In [4]:
import pandas as pd
import time

def read_time_series_with_ananomly(file_path,index_col,timeparse,header_num,ananomly_col):
    timeparsefunction={
        "datetime":from_timestamp_to_datetime,
    }
    dateparse=lambda x:pd.to_datetime(x)
    timeseries=pd.read_csv(file_path,header=header_num)
    timeseries=timeparsefunction.get(timeparse)(timeseries,index_col)
    return timeseries.set_index(index_col)

def from_timestamp_to_datetime(timeseries,index_col):
    timeseries[index_col]=timeseries[index_col].map(lambda x: time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(x)))
    return timeseries



In [5]:
filepath="../../dataset/demo.csv"
data=read_time_series_with_ananomly(file_path=filepath,timeparse="datetime",index_col="timestamp",
                                    header_num=0,ananomly_col="label")

data.index


In [7]:
timeseries = data[data["KPI ID"]=="29dbcce8"]


In [9]:
timeseries.loc[:,"value"]

timestamp
2016-07-25 00:00:00    0.249685
2016-07-25 00:05:00   -0.298967
2016-07-25 00:10:00   -0.436130
2016-07-25 00:15:00   -0.298967
2016-07-25 00:20:00   -0.298967
2016-07-25 00:25:00   -0.230385
2016-07-25 00:30:00   -0.298967
2016-07-25 00:35:00   -0.436130
2016-07-25 00:40:00   -0.298967
2016-07-25 00:45:00   -0.230385
2016-07-25 00:50:00   -0.230385
2016-07-25 00:55:00   -0.367548
2016-07-25 01:00:00   -0.230385
2016-07-25 01:05:00   -0.436130
2016-07-25 01:10:00   -0.367548
2016-07-25 01:15:00   -0.504711
2016-07-25 01:20:00   -0.504711
2016-07-25 01:25:00   -0.641874
2016-07-25 01:30:00   -0.573293
2016-07-25 01:35:00   -0.710456
2016-07-25 01:40:00   -0.779037
2016-07-25 01:45:00   -0.779037
2016-07-25 01:50:00   -0.779037
2016-07-25 01:55:00   -0.916200
2016-07-25 02:00:00   -0.847619
2016-07-25 02:05:00   -0.916200
2016-07-25 02:10:00   -0.916200
2016-07-25 02:15:00   -0.984782
2016-07-25 02:20:00   -1.053363
2016-07-25 02:25:00   -0.984782
                         ...  

In [55]:
import time
time.localtime(data.iloc[0:1,0:0])

TypeError: an integer is required (got type DataFrame)

In [72]:
data.columns

Index(['KPI ID', 'value', 'label'], dtype='object')

AttributeError: 'set' object has no attribute 'map'